In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db    

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type in ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM album
LIMIT 5

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT *
FROM customer
LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


Query that returns each genre, with the number of tracks sold in the USA:
1. in absolute numbers
2. in percentages

In [5]:
%%sql

WITH inv_usa AS
(
    SELECT
        i.invoice_id,
        il.track_id,
        il.unit_price,
        il.quantity,
        g.name AS genre,
        i.total
    FROM invoice i
    JOIN invoice_line il
        ON i.invoice_id = il.invoice_id
    LEFT JOIN track t
        ON t.track_id = il.track_id
    LEFT JOIN genre g
        ON t.genre_id = g.genre_id
    WHERE billing_country = 'USA'
),
usa_total AS
(
    SELECT SUM(quantity) AS usa_sales
    FROM inv_usa
),
usa_genre AS
(
    SELECT 
        genre,
    SUM(quantity) AS qtt_genre
    FROM inv_usa
    GROUP BY genre
)
SELECT
    genre,
    qtt_genre,
    (qtt_genre * 100) / (SELECT usa_sales FROM usa_total)
FROM usa_genre
ORDER BY 2 DESC

Done.


genre,qtt_genre,(qtt_genre * 100) / (SELECT usa_sales FROM usa_total)
Rock,561,53
Alternative & Punk,130,12
Metal,124,11
R&B/Soul,53,5
Blues,36,3
Alternative,35,3
Latin,22,2
Pop,22,2
Hip Hop/Rap,20,1
Jazz,14,1


In terms of genre most sold in the US compared the genres offered by the record label, the order goes
Punk (12%) > Blues (3%) > Pop (2%) > Hip-Hop (1%)

Given this, we'd recommend Red Tone, Slim Jim Bites, and Meteor and the Girls 

Next we'll find the total dollar amount of sales assigned to each sales support agent within the company. We'll also add any relevant attributes for that employee.

In [6]:
%%sql

SELECT
    e.employee_id,
    e.first_name || ' ' || e.last_name AS emp_name,
    e.hire_date,
    e.city,
    e.state,
    e.country,
    SUM(i.total) AS total_sales
FROM invoice i
JOIN customer c on i.customer_id = c.customer_id
JOIN employee e on c.support_rep_id = e.employee_id
GROUP BY 
    e.employee_id,
    e.hire_date,
    e.city,
    e.state,
    e.country
ORDER BY SUM(i.total) DESC

Done.


employee_id,emp_name,hire_date,city,state,country,total_sales
3,Jane Peacock,2017-04-01 00:00:00,Calgary,AB,Canada,1731.5100000000039
4,Margaret Park,2017-05-03 00:00:00,Calgary,AB,Canada,1584.0000000000034
5,Steve Johnson,2017-10-17 00:00:00,Calgary,AB,Canada,1393.920000000002


Jane Peacock has the highest number of sales. This could be attributed to the fact that she joined the company before other support agents.

Next, we will analyze the sales data for customers from each different country. We'll use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we need to calculate data, for each country, on the:

1. total number of customers
2. total value of sales
3. average value of sales per customer
4. average order value


Write a query that collates data on purchases from different countries.
- Where a country has only one customer, collect them into an "Other" group.
- The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
- For each country, include:
-- total number of customers
-- total value of sales
-- average value of sales per customer
-- average order value

In [7]:
%%sql

WITH ct_country AS 
(
    SELECT
        c.country, 
        COUNT(distinct c.customer_id) AS ct_cus,
        SUM(i.total) AS tot_sales,
        COUNT(i.invoice_id) AS tot_orders
    FROM customer c
    JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY country
),
sorted_ct AS
(
    SELECT 
        CASE
            WHEN ct_cus = 1 THEN 'Other'
            ELSE country
        END AS country,
        CASE
            WHEN ct_cus = 1 THEN 1
            ELSE 0
        END AS sort,
        ct_cus,
        tot_sales,
        tot_orders
    FROM ct_country
)
SELECT
    country,
    SUM(ct_cus) AS total_customers,
    ROUND(SUM(tot_sales), 2) AS total_value_sales,
    ROUND(SUM(tot_sales) / SUM(ct_cus), 2) AS avg_sale_cust,
    ROUND(SUM(tot_sales) / SUM(tot_orders), 2) AS avg_order_val
FROM sorted_ct
GROUP BY country
ORDER BY sort ASC, ct_cus DESC




Done.


country,total_customers,total_value_sales,avg_sale_cust,avg_order_val
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


Next, we will review the data to strategize purchasing by Chinook. We'll categorize each invoice as either an album purchase or not, and calculate the following summary statistics:
- Number of invoices
- Percentage of invoices

In [16]:
%%sql 

WITH select_track AS
(
    SELECT
        il.invoice_id invoice_id,
        MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
)
SELECT 
    album_purchase, 
    COUNT(invoice_id),
    COUNT(invo)
    FROM (
        SELECT
            st.*,
            CASE
                WHEN
                     (
                      SELECT t.track_id FROM track t
                      WHERE t.album_id = (
                                          SELECT t2.album_id FROM track t2
                                          WHERE t2.track_id = st.first_track_id
                                         ) 

                      EXCEPT 

                      SELECT il2.track_id FROM invoice_line il2
                      WHERE il2.invoice_id = st.invoice_id
                     ) IS NULL
                 AND
                     (
                      SELECT il2.track_id FROM invoice_line il2
                      WHERE il2.invoice_id = st.invoice_id

                      EXCEPT 

                      SELECT t.track_id FROM track t
                      WHERE t.album_id = (
                                          SELECT t2.album_id FROM track t2
                                          WHERE t2.track_id = st.first_track_id
                                         ) 
                     ) IS NULL
                 THEN "yes"
                 ELSE "no"
             END AS "album_purchase"
         FROM select_track st
    ) 
    GROUP BY album_purchase

Done.


invoice_id,first_track_id,album_purchase
1,1158,yes
2,201,no
3,2516,no
4,748,no
5,1986,yes
6,30,no
7,42,no
8,81,no
9,196,no
10,2663,no
